# Expert Knowledge Assistant

### A specialized Q&A agent designed for expert-level knowledge tasks. Built for use by Insurellm employees, an Insurance Technology company, while prioritizing accuracy while keeping costs low

This project utilizes Retrieval-Augmented Generation (RAG) to ensure the assistant delivers high-accuracy responses.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [3]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [4]:
# Opting for a cost-efficient model due to pricing constraints

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [5]:
# Load environment variables in a files

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
# Load all markdown documents from each subfolder in the knowledge base
# Applies UTF-8 encoding by default (with a fallback option for Windows users if needed)
# Tags each document with its folder name as metadata for later context identification

folders = glob.glob("knowledge-base/*")
text_loader_kwargs = {'encoding': 'utf-8'}
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [7]:
len(documents)

31

In [15]:
documents[23]

Document(metadata={'source': 'knowledge-base\\employees\\Maxine Thompson.md', 'doc_type': 'employees'}, page_content="# HR Record\n\n# Maxine Thompson\n\n## Summary\n- **Date of Birth:** January 15, 1991  \n- **Job Title:** Data Engineer  \n- **Location:** Austin, Texas  \n\n## Insurellm Career Progression\n- **January 2017 - October 2018**: **Junior Data Engineer**  \n  * Maxine joined Insurellm as a Junior Data Engineer, focusing primarily on ETL processes and data integration tasks. She quickly learned Insurellm's data architecture, collaborating with other team members to streamline data workflows.  \n- **November 2018 - December 2020**: **Data Engineer**  \n  * In her new role, Maxine expanded her responsibilities to include designing comprehensive data models and improving data quality measures. Though she excelled in technical skills, communication issues with non-technical teams led to some project delays.  \n- **January 2021 - Present**: **Senior Data Engineer**  \n  * Maxine 

In [ ]:
# Split documents into smaller chunks to improve retrieval and model input handling
# Each chunk is 1000 characters long with a 200-character overlap for context continuity

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [10]:
len(chunks)

123

In [17]:
print(chunks[8])

page_content='# Contract with Belvedere Insurance for Markellm

## Terms
This Contract ("Agreement") is made and entered into as of [Date] by and between Insurellm, Inc., a corporation registered in the United States, ("Provider") and Belvedere Insurance, ("Client"). 

1. **Service Commencement**: The services described herein will commence on [Start Date].
2. **Contract Duration**: This Agreement shall remain in effect for a period of 1 year from the Commencement Date, unless terminated earlier in accordance with the termination clause of this Agreement.
3. **Fees**: Client agrees to pay a Basic Listing Fee of $199/month for accessing the Markellm platform along with a performance-based pricing of $25 per lead generated.
4. **Payment Terms**: Payments shall be made monthly, in advance, with invoices issued on the 1st of each month, payable within 15 days of receipt.' metadata={'source': 'knowledge-base\\contracts\\Contract with Belvedere Insurance for Markellm.md', 'doc_type': 'contra

In [ ]:
# Create a set of unique document types by extracting the 'doc_type' metadata from each chunk
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: products, contracts, employees, company


In [ ]:
# Print chunks that contain the word 'Avery'
# Adds a separator after each match
for chunk in chunks:
    if 'Avery' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='# About Insurellm

Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. It's first product was Markellm, the marketplace connecting consumers with insurance providers.
It rapidly expanded, adding new products and clients, reaching 200 emmployees by 2024 with 12 offices across the US.' metadata={'source': 'knowledge-base\\company\\about.md', 'doc_type': 'company'}
_________
page_content='# Avery Lancaster

## Summary
- **Date of Birth**: March 15, 1985  
- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  
- **Location**: San Francisco, California  

## Insurellm Career Progression
- **2015 - Present**: Co-Founder & CEO  
  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the compan